In [4]:
import numpy as np

# Create a 10x2 array
array_10x2 = np.array([[1, 2],
                       [3, 4],
                       [5, 6],
                       [7, 8],
                       [9, 10],
                       [11, 12],
                       [13, 14],
                       [15, 16],
                       [17, 18],
                       [19, 20]])

# Reshape the array to a 20-element vector
vector_20 = array_10x2.reshape(-1)

# Print the results
print("Original 10x2 array:")
print(array_10x2)
print("\nReshaped 20-element vector:")
print(vector_20)

Original 10x2 array:
[[ 1  2]
 [ 3  4]
 [ 5  6]
 [ 7  8]
 [ 9 10]
 [11 12]
 [13 14]
 [15 16]
 [17 18]
 [19 20]]

Reshaped 20-element vector:
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


In [ ]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib as mpl
from matplotlib.pyplot import cm
import cv2
from tqdm import trange
import computer_vision as cv




def compute_reprojection_error(P_arr, Xj, xj_arr):

    xj_proj_arr = np.array([cv.transform_and_dehomogenize(P, Xj) for P in P_arr])
    res = [np.array([xj_arr[i][0] - xj_proj_arr[i][0], xj_arr[i][1] - xj_proj_arr[i][1]]) for i in xj_proj_arr.shape[0]]
    res = np.array(res).reshape(-1)
    reproj_err = LA.norm(res)**2

    return reproj_err, res

def compute_total_reprojection_error(P1, P2, X_est, x1, x2, verbose=False):

    n_pts = np.size(X_est,1)
    reproj_err_tot = []
    res_tot = []

    for j in range(n_pts):

        Xj = X_est[:,j]
        x1j = x1[:,j]
        x2j = x2[:,j]

        reproj_err, res = compute_reprojection_error(P1, P2, Xj, x1j, x2j)
        reproj_err_tot.append(reproj_err)
        res_tot.append(res)
    
    res_tot = np.concatenate(res_tot, 0)

    if verbose:
        print('\nTotal reprojection error:', round(np.sum(reproj_err_tot), 2))
        print('Median reprojection error:', round(np.median(reproj_err_tot), 2))
        print('Avg. reprojection error:', round(np.mean(reproj_err_tot), 2))

    return reproj_err_tot, res_tot




def compute_jacobian_of_residual_wrt_T(Pi, Xj):
    J1 = (Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2 - (1 / (Pi[-1,:] @ Xj))
    J2 = (Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2 - (1 / (Pi[-1,:] @ Xj))
    J = np.row_stack((J1, J2))
    return J



def linearize_reprojection_error(P_arr, Xj, xj_arr):

    _, r = compute_reprojection_error(P_arr, Xj, xj_arr)

    JT = np.array([compute_jacobian_of_residual_wrt_T(P, Xj) for P in P_arr])
    JT = np.concatenate(JT, 0)
    
    return r, JT 

def compute_update(r, J, mu):
    I = np.eye(np.size(J,1))
    delta = -LA.inv(J.T @ J + mu*I) @ J.T @ r
    return delta



def minimize_reprojection_error(P_arr, X, x_arr, mu_init, n_its, verbose=False):

    n_pts = np.size(X,1)
    ts = []

    for j in trange(n_pts):
        
        Xj = X[:,j]
        xj_arr = x_arr[:,j]

        # R_arr
        # T_arr

        converged = False
        mu = mu_init
        t = 0
    
        while (t <= n_its) and converged is not True:
            t += 1
            
            _, _, w = compute_axis_angle_representation(R)
            r, Jw, JX, JT = linearize_reprojection_error(P_arr, Xj, xj_arr)

            delta_w = compute_update(r, Jw, mu)
            w_opt = w + delta_w
            R_opt = compute_R_matrix_exponential(w_opt, approx=True) @ R
            U, _, VT = LA.svd(R_opt)
            R_opt = U @ VT
            
            delta_Xj = compute_update(r, JX, mu)
            Xj_opt = cv.dehomogenize(Xj + delta_Xj)

            delta_T = compute_update(r, JT, mu)
            T_opt = T + delta_T
            
            reproj_err, _ = compute_reprojection_error(P_arr, Xj, xj_arr)
            reproj_err_opt, _ = compute_reprojection_error(P_arr, Xj_opt, xj_arr)

            if np.isclose(reproj_err_opt, reproj_err):
                converged = True
            elif reproj_err_opt < reproj_err:
                Xj = Xj_opt
                R = np.copy(R_opt)
                T = np.copy(T_opt)
                mu /= 10
            else:
                mu *= 10
        
        X[:,j] = Xj
        ts.append(t)

    if verbose:
        print('\nAvg its:', np.mean(ts))
        print('Max its:', np.max(ts))
        print('Min its:', np.min(ts))

    return X







import scipy.optimize

def compute_residual(params, x):

    w = params[0]
    T = params[1]
    X = params[2]
    
    R = compute_R_matrix_exponential(w)
    x_proj = cv.dehomogenize(R @ X[:-1] + T)
    residual = x_proj - x

    return residual

w = compute_axis_angle_representation(R)
x0 = np.array([w, T, X])
result = optimize.least_squares(compute_residual, x0, method='lm', args=(x)) 

